# Введение в анализ данных
## НИУ ВШЭ, 2019-2020 учебный год

### Домашнее задание №3

Задание выполнил: Кувайцев Константин

### Общая информация
Дата выдачи: 

Дедлайн: 

### О задании

В этом домашнем задании вы будете работать с линейной классификацией, попрактикуетесь на реальной задаче классификации текстов.

Для решения этого домашнего задания намного удобнее будет использовать Colab, так как данных много.

### Оценивание и штрафы

За сдачу задания позже срока на итоговую оценку за задание накладывается штраф в размере 1 балл в день, но получить отрицательную оценку нельзя.

__Внимание!__ Домашнее задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов.

### Формат сдачи
Загрузка файлов с решениями происходит в системе Anytask.

### Подготовка данных

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from nltk.tokenize import WordPunctTokenizer

In [2]:
# Датасет можно скачать здесь

!wget https://www.dropbox.com/s/tg55q9mrziroyrs/train_subset.csv

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


### Данные

Мы имеем дело с данными с торговой платформы Avito.
Для каждого товара представлены следующие параметры:
 - title
 - description
 - Category_name
 - Category

Имеется информация об объектах 50 классов.
Задача: по новым объектам (title, description) предсказать Category.
(Очевидно, что параметр Category_name для предсказания классов использовать нельзя)

In [3]:
data = pd.read_csv("train_subset.csv", index_col='id')

data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [4]:
data.shape

(30000, 4)

In [5]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()

del data

Сразу разделим выборку на train и test.
Никакие данные из test для обучения использовать нельзя!

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
X_train[:5]

del X, y

### Токенизация (1 балл)


Токенизация -- разбиение текста на мелкие части, которые можно обработать машинными методами.
Можно использовать разные алгоритмы токенизации.
Давайте пока остановимся на простом WordPunctTokenizer.


In [8]:
tokenizer = WordPunctTokenizer()


def preprocess(text: str) -> str:
    return ' '.join(tokenizer.tokenize(text.lower()))


text = 'Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...'
print("before:", text,)
print("after:", preprocess(text),)

before: Здраствуйте. Я, Кирилл. Хотел бы чтобы вы сделали игру, 3Д-экшон суть такова...
after: здраствуйте . я , кирилл . хотел бы чтобы вы сделали игру , 3д - экшон суть такова ...


__Задание:__ Токенизируйте title и description в train и test

In [9]:
vect_preprocess = np.vectorize(preprocess)
X_train = vect_preprocess(X_train)
X_test = vect_preprocess(X_test)

In [10]:
X_train[:10]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п .

In [11]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

### BOW (1.5 балла)

Один из традиционных подходов -- построение bag of words.

Метод состоит в следующем:

 - Составить словарь самых часто встречающихся слов в train data
 - Для каждого примера из train посчитать, сколько раз каждое слово из словаря в нём встречается


 В sklearn есть CountVectorizer, но в этом задании его использовать нельзя.

__Задание:__ Найдите k самых частых слов, отсортируйте их по убыванию частотности (k=10000)

In [12]:
bow_vocabulary = {}


def freq_words(string):
    words = string.split(' ')
    for word in words:
        bow_vocabulary[word] = bow_vocabulary.get(word, 0) - 1
    return 0


vect_freq_words = np.vectorize(freq_words)
vect_freq_words(X_train)
bow_vocabulary = list(bow_vocabulary.items())
bow_vocabulary.sort(key=lambda i: i[1])
bow_vocabulary = np.array(bow_vocabulary)
words_counts = bow_vocabulary.copy()

In [13]:
bow_vocabulary = bow_vocabulary[0:10000, 0]


In [14]:
assert sorted(bow_vocabulary)[::200] == ['!', '12500', '270', '700', 'by', 'gh', 'michael', 'sonata', 'ø', 'аудиоподготовка', 'большим', 'веса', 'воспроизведения', 'габариты', 'гтд', 'джинсами', 'доступность', 'загрузки', 'зимней', 'использовался', 'квартала', 'коммуникации', 'кошки', 'лакированные', 'магазин', 'металл', 'мск', 'натуральным', 'носке', 'одному', 'отвечаем', 'пассат', 'плотно', 'покраску', 'постоянные', 'примеры', 'просьба', 'размещайте', 'репетитор', 'сантехник', 'сидения', 'современного', 'стала', 'схема', 'тон', 'удлиненная', 'фасад', 'цветами', 'шея', 'эту']

In [15]:
data = {}
index = 0

def list_to_dict(word_count):
    global index
    data[word_count[0]] = (-int(word_count[1]), index) 
    index += 1
    return 0


np.apply_along_axis(list_to_dict, 1, words_counts[0:10000])

array([0, 0, 0, ..., 0, 0, 0])

In [16]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    """ 
    answer = [0] * 10000
    words = text.split(' ')
    
    for word in words:
        if word in data:
            answer[data[word][1]] += 1

    
    return np.array(answer)

In [17]:
assert np.allclose(np.where(text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml") != 0)[0],
                   np.array([   1,    4,   12,  565,  866, 1601, 2539, 4063])
)

In [18]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description
    answer = [0] * len(items)
    
    for i in range(len(items)):
        answer[i] = text_to_bow(items[i][1])
        
    return np.array(answer)

In [19]:
assert np.allclose(np.where(items_to_bow([X_train[42]])[0] != 0),
                   np.array([   0, 1, 2, 5, 6, 7, 12, 27, 41, 49, 110,
                                189,  208,  221, 2032, 3052, 7179, 9568]),
)

In [20]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

In [21]:
from sklearn.metrics import accuracy_score

### Логистическая регрессия и SVC (1 балл)


Теперь описание каждого товара представлено, как точка в многомерном пространстве.
Очень важно запомнить эту идею: дальше мы будем рассматривать разные способы перехода от текста к точке в пространстве.

Для BOW каждое измерение в пространстве -- какое-то слово.
Мы предполагаем, что текст описывается набором каких-то популярных слов, которые в нём встречаются, а близкие по смыслу тексты будут использовать одинаковые слова.

Обучите логистическую регрессию и SVC с базовыми параметрами.


In [22]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow, y_train)
lr_acc = accuracy_score(bow_model.predict(X_test_bow), y_test)
print('Accuracy LR =', lr_acc)
assert accuracy_score(bow_model.predict(X_test_bow), y_test) > 0.7

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy LR = 0.7011111111111111


In [23]:
from sklearn.svm import LinearSVC

bow_model = LinearSVC(max_iter=70).fit(X_train_bow, y_train)
svc_acc = accuracy_score(bow_model.predict(X_test_bow), y_test)
print('Accuracy SVC =', svc_acc)
assert accuracy_score(bow_model.predict(X_test_bow), y_test) > 0.68

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy SVC = 0.6842222222222222


### Модификация признаков (0.5 балла)

Добавьте title товара в bow с произвольным весом, как изменится качество?

In [0]:
def new_text_to_bow(item: np.array) -> np.array:

    answer = [0] * 10000
    for i in range(2):
        words = item[i].split(' ')

        for word in words:
            if word in data:
                answer[data[word][1]] += 1

    
    return np.array(answer)

In [0]:
def new_items_to_bow(items: np.array) -> np.array:

    answer = [0] * len(items)
    
    for i in range(len(items)):
        answer[i] = new_text_to_bow(items[i])
        
    return np.array(answer)

### Добавление title

In [0]:
X_train_bow_title = new_items_to_bow(X_train)
X_test_bow_title = new_items_to_bow(X_test)
del X_train_bow, X_test_bow

In [28]:
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow_title, y_train)
lr_acc_title = accuracy_score(bow_model.predict(X_test_bow_title), y_test)
print('Accuracy LR после добавления title =', lr_acc_title)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy LR после добавления title = 0.7812222222222223


In [29]:
bow_model = LinearSVC(max_iter=70).fit(X_train_bow_title, y_train)
svc_acc_title = accuracy_score(bow_model.predict(X_test_bow_title), y_test)
print('Accuracy SVC после добавления title =', svc_acc_title)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy SVC после добавления title = 0.7543333333333333


### Нормировка

Нормализуйте данные (`sklearn.preprocessing.normalize`) перед обучением. Что станет с качеством и почему?

In [0]:
from sklearn.preprocessing import normalize

In [0]:
norm_X_train_bow = normalize(X_train_bow_title)
norm_X_test_bow = normalize(X_test_bow_title)

In [34]:
bow_model = LogisticRegression(max_iter=100).fit(norm_X_train_bow, y_train)
norm_lr_acc = accuracy_score(bow_model.predict(norm_X_test_bow), y_test)
print('Accuracy LR после нормировки =', norm_lr_acc)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy LR после нормировки = 0.677


In [35]:
bow_model = LinearSVC(max_iter=70).fit(norm_X_train_bow, y_train)
norm_svc_acc = accuracy_score(bow_model.predict(norm_X_test_bow), y_test)
print('Accuracy SVC после нормировки =', norm_svc_acc)

Accuracy SVC после нормировки = 0.7987777777777778


In [0]:
del norm_X_train_bow, norm_X_test_bow, X_train_bow_title, X_test_bow_title

### mystem (0.5) балла

Попробуйте обучиться, используя токенизатор mystem. Сравните качество.

In [37]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

--2020-04-19 13:52:51--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.241, 5.45.205.245, 5.45.205.244, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.241|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm906.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-04-19 13:52:52--  http://cache-mskm906.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm906.cdn.yandex.net (cache-mskm906.cdn.yandex.net)... 5.45.220.16, 2a02:6b8:0:2002::17
Connecting to cache-mskm906.cdn.yandex.net (cache-mskm906.cdn.yandex.net)|5.45.220.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.1’

mystem-3.0-linux3.1 100%[===================>] 

In [38]:
pip install git+https://github.com/nlpub/pymystem3

  Cloning https://github.com/nlpub/pymystem3 to /tmp/pip-req-build-ihhn4cn2
  Running command git clone -q https://github.com/nlpub/pymystem3 /tmp/pip-req-build-ihhn4cn2
  Created wheel for pymystem3: filename=pymystem3-0.2.0-cp36-none-any.whl size=9921 sha256=ff0470e6f443ed83d92d7998465ce0aad5e7cfd57b23816e1c102384e41338cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-08eeut7g/wheels/7d/75/c2/216a594291dee680749ce12c60d16125cfe1f363059e7163dc
Successfully built pymystem3


In [0]:
from pymystem3 import Mystem

tokenizer = Mystem()

In [0]:
def preprocess_mystem(text: str) -> str:
    return ' '.join(tokenizer.lemmatize(text.lower()))


vect_preprocess_mystem = np.vectorize(preprocess_mystem)

In [0]:
X_train_mystem = vect_preprocess_mystem(X_train)
X_test_mystem = vect_preprocess_mystem(X_test)

In [0]:
bow_vocabulary_mystem = {}


def freq_words_mystem(string):
    words = string.split()
    for word in words:
        bow_vocabulary_mystem[word] = bow_vocabulary_mystem.get(word, 0) - 1
    return 0


vect_freq_words_mystem = np.vectorize(freq_words_mystem)
vect_freq_words_mystem(X_train)
bow_vocabulary_mystem = list(bow_vocabulary_mystem.items())
bow_vocabulary_mystem.sort(key=lambda i: i[1])
bow_vocabulary_mystem = np.array(bow_vocabulary_mystem)
words_counts_mystem = bow_vocabulary_mystem.copy()
bow_vocabulary_mystem = bow_vocabulary_mystem[0:10000, 0]

In [43]:
data_mystem = {}
i = 0

def list_to_dict_mystem(word_count):
    global i
    data_mystem[word_count[0]] = (-int(word_count[1]), i) 
    i += 1
    return 0


np.apply_along_axis(list_to_dict_mystem, 1, words_counts_mystem[0:10000])

array([0, 0, 0, ..., 0, 0, 0])

In [0]:
def text_to_bow_mystem(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    """ 
    answer = [0] * 10000
    words = text.split(' ')
    
    for word in words:
        if word in data_mystem:
            answer[data_mystem[word][1]] += 1

    
    return np.array(answer)

In [0]:
def items_to_bow_mystem(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description
    answer = [0] * len(items)
    
    for i in range(len(items)):
        answer[i] = text_to_bow_mystem(items[i][1])
        
    return np.array(answer)

### Mystem без нормировки и title

In [0]:
X_train_bow_mystem = items_to_bow_mystem(X_train_mystem)
X_test_bow_mystem = items_to_bow_mystem(X_test_mystem)

In [47]:
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow_mystem, y_train)
lr_acc_mystem = accuracy_score(bow_model.predict(X_test_bow_mystem), y_test)
print('Accuracy LR =', lr_acc_mystem)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy LR = 0.7082222222222222


In [48]:
bow_model = LinearSVC(max_iter=70).fit(X_train_bow_mystem, y_train)
svc_acc_mystem = accuracy_score(bow_model.predict(X_test_bow_mystem), y_test)
print('Accuracy SVC =', svc_acc_mystem)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy SVC = 0.6896666666666667


### Mystem с добавлением title

In [0]:
del X_train_bow_mystem, X_test_bow_mystem
X_train_bow_title_mystem = new_items_to_bow(X_train_mystem)
X_test_bow_title_mystem = new_items_to_bow(X_test_mystem)

In [50]:
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow_title_mystem, y_train)
lr_acc_title_mystem = accuracy_score(bow_model.predict(X_test_bow_title_mystem), y_test)
print('Accuracy LR после добавления title =', lr_acc_title_mystem)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy LR после добавления title = 0.7781111111111111


In [51]:
bow_model = LinearSVC(max_iter=70).fit(X_train_bow_title_mystem, y_train)
svc_acc_title_mystem = accuracy_score(bow_model.predict(X_test_bow_title_mystem), y_test)
print('Accuracy SVC после добавления title =', svc_acc_title_mystem)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy SVC после добавления title = 0.7463333333333333


### Mystem нормировка

In [0]:
norm_X_train_bow_mystem = normalize(X_train_bow_title_mystem)
norm_X_test_bow_mystem = normalize(X_test_bow_title_mystem)

In [53]:
bow_model = LogisticRegression(max_iter=100).fit(norm_X_train_bow_mystem, y_train)
norm_lr_acc_mystem = accuracy_score(bow_model.predict(norm_X_test_bow_mystem), y_test)
print('Accuracy LR после нормировки =', norm_lr_acc_mystem)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy LR после нормировки = 0.6906666666666667


In [54]:
bow_model = LinearSVC(max_iter=70).fit(norm_X_train_bow_mystem, y_train)
norm_svc_acc_mystem = accuracy_score(bow_model.predict(norm_X_test_bow_mystem), y_test)
print('Accuracy SVC после нормировки =', norm_svc_acc_mystem)

Accuracy SVC после нормировки = 0.7838888888888889


In [0]:
LR_acc = {'LR acc': [lr_acc, lr_acc_mystem], 'LR acc title': [lr_acc_title, lr_acc_title_mystem], 
      'LR acc norm': [norm_lr_acc, norm_lr_acc_mystem]}
SVC_acc = {'SVC acc': [svc_acc, svc_acc_mystem], 'SVC acc title': [svc_acc_title, svc_acc_title_mystem], 
      'SVC acc norm': [norm_svc_acc, norm_svc_acc_mystem]}

In [56]:
data_lr_acc = pd.DataFrame(LR_acc)
data_svc_acc = pd.DataFrame(SVC_acc)
data_lr_acc

,LR acc,LR acc title,LR acc norm
0,0.701111,0.781222,0.677000
1,0.708222,0.778111,0.690667


In [57]:
data_svc_acc

,SVC acc,SVC acc title,SVC acc norm
0,0.684222,0.754333,0.798778
1,0.689667,0.746333,0.783889


Как видно из таблиц наилучшие показатели имеют 
1) модель LogisticRegression, обученная на данных с добавлением title при помощи WordPunctTokenizer
2) модель SVM, обученная на нормированых данных при помощи WordPunctTokenizer

In [0]:
del norm_X_train_bow_mystem, norm_X_test_bow_mystem, X_train_bow_title_mystem, X_test_bow_title_mystem

### TF-IDF (1.5 балла)

Не все слова полезны одинаково, давайте попробуем [взвесить](http://tfidf.com/) их, чтобы отобрать более полезные.


> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
> 
> IDF(t) = log_e(Total number of documents / Number of documents with term t in it).


В sklearn есть TfidfVectorizer, но в этом задании его использовать нельзя.

In [59]:
# Давайте для простоты считать один tf-idf для title и description.
# Для каждого слова из bow_vocabulary нужно посчитать
# в тексте скольких товаров встретилось это слово

def idf_find(items: np.array) -> np.array:
    answer = [0] * 10000
    
    for item in items:
        item_words = [True] * 10000
        
        for t_d in item:
            words = t_d.split(' ')
            
            for word in words:
                if word in data and item_words[data[word][1]]:
                    answer[data[word][1]] += 1
                    item_words[data[word][1]] = False
                    
    answer = np.log(np.array(answer) / 30000)
    return answer
    
idf = idf_find(X_train)
idf

array([-1.16903475, -0.71812317, -0.64530985, ..., -7.91105739,
       -8.00636757, -8.69951475])

In [0]:
def items_to_tfidf(items: np.array) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указан tf-idf
    """
    tfidf = []
    
    for item in items:
        tf= [0] * 10000
        
        for t_d in item:
            words = t_d.split(' ')

            for word in words:
                if word in data:
                    tf[data[word][1]] += 1

        tf = np.array(tf)
        tfidf.append(tf * idf)
        
    return tfidf
    

In [0]:
X_train_tfidf = items_to_tfidf(X_train)
X_test_tfidf = items_to_tfidf(X_test)

In [0]:
# Нормализуйте данные
from sklearn.preprocessing import normalize


X_train_tfidf_norm = normalize(X_train_tfidf)
X_test_tfidf_norm = normalize(X_test_tfidf)

### Модели на TF-IDF признаках (1 балл)

Обучите логистическую регрессию и SVC, оцените качество (accuracy_score)

In [63]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression(max_iter=100).fit(X_train_tfidf_norm, y_train)
lr_acc_tfidf = accuracy_score(bow_model.predict(X_test_tfidf_norm), y_test)
print('Accuracy LR с tfidf =', lr_acc_tfidf)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy LR с tfidf = 0.7627777777777778


In [64]:
from sklearn.svm import LinearSVC
bow_model = LinearSVC(max_iter=70).fit(X_train_tfidf_norm, y_train)
svc_acc_tfidf = accuracy_score(bow_model.predict(X_test_tfidf_norm), y_test)
print('Accuracy SVC с tfidf =', svc_acc_tfidf)

Accuracy SVC с tfidf = 0.8074444444444444


### Hashing Vectorizer (0.5 балла)

Попробуйте использовать `sklearn.feature_extraction.text.HashingVectorizer` для векторизации текстов.
Обязательно оцените качество работы алгоритмов классификации с использованием новой векторизации.

In [0]:
# Your code here

### Word Vectors (1 балл)

Давайте попробуем другой подход -- кажому слову сопоставим какой-то эмбеддинг (вектор).

Вектора будут небольшой размерности. Таким образом мы снизим количество параметров в модели.

Вектора мы возьмём уже готовые (обученные на текстах их интернета), так что наша модель будет знать некоторую дополнительную информацию о внешнем мире.

In [65]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

--2020-04-19 14:18:58--  https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0x7oxso6x93efzj/ru.tar.gz [following]
--2020-04-19 14:18:58--  https://www.dropbox.com/s/raw/0x7oxso6x93efzj/ru.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbc0afbc63adecea93733558c1a.dl.dropboxusercontent.com/cd/0/inline/A2Ly3RvRjdlilwq_IFF9SDrSVZg39zkf8Gm87F0h18STevUgcRYDDpmYH56P5sObYfyIsZwQRIsamoTe0Dfe7Qj1pNmZQ7y5xtaTaSKJtUuamw/file# [following]
--2020-04-19 14:18:59--  https://ucbc0afbc63adecea93733558c1a.dl.dropboxusercontent.com/cd/0/inline/A2Ly3RvRjdlilwq_IFF9SDrSVZg39zkf8Gm87F0h18STevUgcRYDDpmYH56P5sObYfyIsZwQRIsamoTe0Dfe7Qj1pNmZQ7y5xtaTaSKJtUuamw/file
Resolving ucbc0afbc63adecea9

In [66]:
!tar -xzvf ru.tar.gz

ru.bin
ru.vec


In [0]:
import gensim
from gensim.models.wrappers import FastText


model = FastText.load_fasttext_format('ru.bin')

In [0]:
# Эмбеддинг предложения -- сумма эмбеддингов токенов


def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """

    embedding_dim = model['кек'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')
    
    for word in sentence.split():
        if word in model:
            features += model[word]
    
    return features

In [0]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

In [0]:
def items_embedding(items: np.array) -> np.array:
    answer = []
    for item in items:
        answer.append(sentence_embedding(item[1]))

    return answer

In [0]:
X_train_emb = items_embedding(X_train)
X_test_emb = items_embedding(X_test)

In [73]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression(max_iter=100).fit(X_train_emb, y_train)
lr_acc_emb = accuracy_score(bow_model.predict(X_test_emb), y_test)
print('Accuracy LR с embedding =', lr_acc_emb)

Accuracy LR с embedding = 0.4941111111111111


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [74]:
from sklearn.svm import LinearSVC
bow_model = LinearSVC(max_iter=70).fit(X_train_emb, y_train)
svc_acc_emb = accuracy_score(bow_model.predict(X_test_emb), y_test)
print('Accuracy SVC с embedding =', svc_acc_emb)

Accuracy SVC с embedding = 0.4702222222222222


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### Что дальше?

Решение каждого пункта 1 балл:

1. N-Gram модели текстовой классификации

2. Обучиться на полных данных (контест на kaggle)

3. Поработать с другими эмбеддингами (word2vec, GloVe).

4. Использовать Vowpal Wabbit вместо sklearn.

5. Другие способы токенизации (pymorphy2, spaCy)


Снабжайте код пояснениями и графиками.
Обязательно необходимо написать вывод по каждому пункту, который вы реализуете.